In [1]:
import sys
sys.path.append("..")
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from scipy.interpolate import interp1d
import os
from script.recovery import RocketParachuteCalculator
# import sys

# sys.path.append("..")


ParaCalc = RocketParachuteCalculator()

## Manual


In [2]:
DATA_DIRECTORY = "data"
AIR_PROPERTIES_DATA_FILE_NAME = "air_values.csv"
ITEM_LIST_DATA_FILE_NAME = "item_list.csv"

# GENERAL DATA
MASS_OF_ROCKET = 25  # kg
AIR_TEMP_CELSIUS = 25  # Celsius

# MAIN
NUMBER_OF_MAINS = 1
NUMBER_OF_REDUNDANT_MAINS = 0
MAIN_MARGIN_FACTOR = 1.2
MAIN_REQUIRED_DESCENT_VELOCITY = 5

#DROGUE
NUMBER_OF_DROGUES = 1
NUMBER_OF_REDUNDANT_DROGUES = 0
DROGUE_MARGIN_FACTOR = 1
DROGUE_REQUIRED_DESCENT_VELOCITY = 30
# ROUND PARAMETERS
LENGTH_TO_HALF_CIRCUMFERENCE_RATIO = 1
HOLE_TO_CANOPY_AREA_RATIO = 0.04
MASS_PER_UNIT_AREA = 3.5 # ounce/square yard
NUMBER_OF_GORES_PANELS = [4, 4, 4, 4, 4, 12, 12, 12, 12, 12]
RATIO_OF_HEIGHT_TO_RADIUS = [1, 0.9, 0.8, 0.7, 0.6, 1, 0.9, 0.8, 0.7, 0.6]
DRAG_COEFFICIENT = 1.5

### Air Properties


In [3]:
# Get the current working directory (should be 'Notebooks')
cwd = os.getcwd()
# Go up one level to the project directory
project_dir = os.path.join(cwd, '..')
# Construct the path to the CSV file in the 'data' directory
air_csv_path = os.path.join(project_dir, DATA_DIRECTORY,
                            AIR_PROPERTIES_DATA_FILE_NAME)
# Normalize the path (optional but recommended)
air_csv_path = os.path.normpath(air_csv_path)

item_csv_path = os.path.join(project_dir, DATA_DIRECTORY,
                             ITEM_LIST_DATA_FILE_NAME)
# Normalize the path (optional but recommended)
item_csv_path = os.path.normpath(item_csv_path)

air_prop_df = pd.read_csv(air_csv_path)
item_list_df = pd.read_csv(item_csv_path)


# display(air_prop_df)

In [4]:
rename_cols = {
    'Temperature\n(K)': 'temperature',
    'Specific Heat': 'specific_heat_cp',
    'Unnamed: 2': 'specific_heat_cv',
    'Ratio of Specific Heats\n- k -\n(cp/cv)': 'ratio_of_specific_heats',
    'Dynamic Viscosity\n- ? -\n(10-5 kg/m s)': 'dynamic_viscosity',
    'Thermal Conductivity\n(10-5 kW/m K)': 'thermal_conductivity',
    'Prandtl Number': 'prandtl_number',
    'Kinematic Viscosity1)\n- ? -\n(10-5 m2/s)': 'kinematic_viscosity',
    'Density1)\n- ? -\n(kg/m3)': 'density',
    'Diffusivity\n- ? - \n(10-6 m2/s)': 'diffusivity'
}

air_prop_df.rename(columns=rename_cols, inplace=True)
# Drop the first row (row 0)
air_prop_df = air_prop_df.drop(0)

# Reset the index
air_prop_df = air_prop_df.reset_index(drop=True)

In [5]:
display(air_prop_df)

,temperature,specific_heat_cp,specific_heat_cv,ratio_of_specific_heats,dynamic_viscosity,thermal_conductivity,prandtl_number,kinematic_viscosity,density,diffusivity
0,175.0,1.0023,0.7152,1.401,1.182,1.593,0.744,0.586,2.0170,NaN
1,200.0,1.0025,0.7154,1.401,1.329,1.809,0.736,0.753,1.7650,10.17
2,225.0,1.0027,0.7156,1.401,1.467,2.020,0.728,0.935,1.5690,NaN
3,250.0,1.0031,0.716,1.401,1.599,2.227,0.720,1.132,1.4120,15.67
4,275.0,1.0038,0.7167,1.401,1.725,2.428,0.713,1.343,1.2840,NaN
5,300.0,1.0049,0.7178,1.400,1.846,2.624,0.707,1.568,1.1770,22.07
6,325.0,1.0063,0.7192,1.400,1.962,2.816,0.701,1.807,1.0860,NaN
7,350.0,1.0082,0.7211,1.398,2.075,3.003,0.697,2.056,1.0090,29.18
8,375.0,1.0106,0.7235,1.397,2.181,3.186,0.692,2.317,0.9413,NaN
9,400.0,1.0135,0.7264,1.395,2.286,3.365,0.688,2.591,0.8824,36.94


## Item List


In [6]:
display(item_list_df)

,object,category,mass_per_unit,cost_per_unit,number,amount_per_pack,total_amount,sub_total,comment
0,Ripstop nylon,fabric,NaN,6.00,5.2,m^2,NaN,267.434473,148x100cm
1,Parachute line,items,NaN,5.48,2.0,15m,30m,10.960000,30m per unit
2,Small quick (chain) links,items,NaN,8.80,3.0,2,6,3.500000,2 per pack
3,(Fishing) bearing swivels,items,NaN,5.50,4.0,1,4,22.000000,1000lb
4,Cut proof tether,items,NaN,6.85,2.0,1m,2,13.700000,NaN
5,High strength paracord,items,NaN,9.98,1.0,30m,30m,9.980000,30m per unit
6,Shock cord,items,NaN,20.00,1.0,20m,20m,20.000000,NaN
7,"Fireproof cloth, large",items,NaN,12.00,2.0,1,2,24.000000,12x12 in
8,"Fireproof cloth, small",items,NaN,10.00,2.0,1,2,20.000000,9x9 in
9,"Thimbles, 4mm",items,NaN,2.48,2.0,4,8,4.960000,NaN


## Chute Main Panel


### General Data


#### MANUAL


#### Density Interpolation


In [7]:
air_temp_kelvin = AIR_TEMP_CELSIUS+273.15

interpolation_function = interp1d(
    air_prop_df["temperature"], air_prop_df["density"], kind='linear', fill_value="extrapolate")

# Use the interpolation function to find the density at the target temperature
density_of_air = interpolation_function(air_temp_kelvin)
print(density_of_air)

1.1849180000000001


### Mains


#### General Data


In [8]:
main_area_cd = ParaCalc.calc_area_cd(safety_factor=MAIN_MARGIN_FACTOR, rocket_mass=MASS_OF_ROCKET,
                                air_density=density_of_air, descent_velocity=MAIN_REQUIRED_DESCENT_VELOCITY)

print(main_area_cd)

19.86972938211758


### Drogue

#### General Data

In [9]:
drogue_area_cd = ParaCalc.calc_area_cd(safety_factor=DROGUE_MARGIN_FACTOR, rocket_mass=MASS_OF_ROCKET,
                                     air_density=density_of_air, descent_velocity=DROGUE_REQUIRED_DESCENT_VELOCITY)
print(drogue_area_cd)

0.4599474394008699


## Round Parameters


In [10]:
mass_per_unit_area_SI = MASS_PER_UNIT_AREA * \
    0.03  # kgm/m^2

cost_per_unit_area = item_list_df.loc[item_list_df['object']
                                      == 'Ripstop nylon', 'cost_per_unit'].values[0]

print(cost_per_unit_area)

6.0


#### Main


In [13]:
side_length = ParaCalc.calc_gore_panel_side_length(
    area_cd=main_area_cd, hole_to_canopy_area_ratio=HOLE_TO_CANOPY_AREA_RATIO, drag_coefficient=DRAG_COEFFICIENT, number_of_gores_panels=NUMBER_OF_GORES_PANELS)

circumference = ParaCalc.calc_circumference(
    area_cd=main_area_cd, hole_to_canopy_area_ratio=HOLE_TO_CANOPY_AREA_RATIO, drag_coefficient=DRAG_COEFFICIENT)

radius = ParaCalc.calc_radius(circumference=circumference)

semi_ellipsoid_depth = ParaCalc.calc_semi_ellipsoid_depth(
    radius=radius, ratio_of_height_to_radius=RATIO_OF_HEIGHT_TO_RADIUS)

length_of_curve = ParaCalc.calc_length_of_curve(
    radius=radius, semi_ellipsoid_depth=semi_ellipsoid_depth, length_to_half_circumference_ratio=LENGTH_TO_HALF_CIRCUMFERENCE_RATIO)

length_of_curve_without_hole = ParaCalc.calc_length_of_curve_without_hole(
    length_of_curve=length_of_curve, hole_to_canopy_area_ratio=HOLE_TO_CANOPY_AREA_RATIO)

circular_area_of_selected = ParaCalc.calc_circular_area(
    area_cd=main_area_cd, drag_coefficient=DRAG_COEFFICIENT, hole_to_canopy_area_ratio=HOLE_TO_CANOPY_AREA_RATIO)

circular_diameter_of_selected = ParaCalc.calc_circular_diameter(
    circular_area_of_selected=circular_area_of_selected)

material_area_per_gore = ParaCalc.calc_material_area_per_gore(
    side_length=side_length, length_of_curve_without_hole=length_of_curve_without_hole)

material_area_per_chute = ParaCalc.calc_material_area_per_chute(
    number_of_gores_panels=NUMBER_OF_GORES_PANELS, material_area_per_gore=material_area_per_gore)

material_area_for_net = ParaCalc.calc_material_area_for_net(
    number_of_gores_panels=NUMBER_OF_GORES_PANELS, length_of_curve_without_hole=length_of_curve_without_hole, side_length=side_length)

mass_per_chute = ParaCalc.calc_mass_per_chute(
    material_area_per_chute=material_area_per_chute, mass_per_unit_area=mass_per_unit_area_SI)

cost_per_chute = ParaCalc.calc_cost_per_chute(
    material_area_for_net=material_area_for_net, cost_per_unit_area=cost_per_unit_area)



# print(side_length)
# print(circumference)
# print(radius)
# print(semi_ellipsoid_depth)
# print(length_of_curve)
# print(length_of_curve_without_hole)

# print(circular_area_of_selected)
# print(circular_diameter_of_selected)
# print(material_area_per_gore)
# print(material_area_per_chute)
# print(material_area_for_net)
# print(mass_per_chute)
# print(cost_per_chute)

In [19]:
round_parameters_dict = {
    'number_of_gores_panels': NUMBER_OF_GORES_PANELS,
    'ratio_of_height_to_radius': RATIO_OF_HEIGHT_TO_RADIUS,
    'drag_coefficient': DRAG_COEFFICIENT,
    'side_length': side_length,
    'circumference': circumference,
    'radius': radius,
    'length_of_curve': length_of_curve,
    'semi_ellipsoid_depth': semi_ellipsoid_depth,
    'length_of_curve_without_hole': length_of_curve_without_hole,
    'material_area_per_gore': material_area_per_gore,
    'material_area_per_chute': material_area_per_chute,
    'material_area_for_net': material_area_for_net,
    'mass_per_chute': mass_per_chute,
    'cost_per_chute': cost_per_chute
}


# Convert the dictionary to a DataFrame

round_parameters_main_df = pd.DataFrame(round_parameters_dict)

display(round_parameters_main_df)

,number_of_gores_panels,ratio_of_height_to_radius,drag_coefficient,side_length,circumference,radius,length_of_curve,semi_ellipsoid_depth,length_of_curve_without_hole,material_area_per_gore,material_area_per_chute,material_area_for_net,mass_per_chute,cost_per_chute
0,4,1.0,1.5,0.500862,2.003447,0.318858,0.500862,0.318858,0.480827,0.153316,0.613263,0.963312,0.064393,5.779870
1,4,0.9,1.5,0.500862,2.003447,0.318858,0.476148,0.286973,0.457102,0.145751,0.583004,0.915780,0.061215,5.494680
2,4,0.8,1.5,0.500862,2.003447,0.318858,0.452168,0.255087,0.434081,0.138410,0.553642,0.869658,0.058132,5.217951
3,4,0.7,1.5,0.500862,2.003447,0.318858,0.429053,0.223201,0.411891,0.131335,0.525340,0.825202,0.055161,4.951213
4,4,0.6,1.5,0.500862,2.003447,0.318858,0.406975,0.191315,0.390696,0.124577,0.498307,0.782738,0.052322,4.696429
5,12,1.0,1.5,0.166954,2.003447,0.318858,0.500862,0.318858,0.480827,0.051105,0.613263,0.963312,0.064393,5.779870
6,12,0.9,1.5,0.166954,2.003447,0.318858,0.476148,0.286973,0.457102,0.048584,0.583004,0.915780,0.061215,5.494680
7,12,0.8,1.5,0.166954,2.003447,0.318858,0.452168,0.255087,0.434081,0.046137,0.553642,0.869658,0.058132,5.217951
8,12,0.7,1.5,0.166954,2.003447,0.318858,0.429053,0.223201,0.411891,0.043778,0.525340,0.825202,0.055161,4.951213
9,12,0.6,1.5,0.166954,2.003447,0.318858,0.406975,0.191315,0.390696,0.041526,0.498307,0.782738,0.052322,4.696429


### Drogue

In [15]:
side_length = ParaCalc.calc_gore_panel_side_length(
    area_cd=drogue_area_cd, hole_to_canopy_area_ratio=HOLE_TO_CANOPY_AREA_RATIO, drag_coefficient=DRAG_COEFFICIENT, number_of_gores_panels=NUMBER_OF_GORES_PANELS)

circumference = ParaCalc.calc_circumference(
    area_cd=drogue_area_cd, hole_to_canopy_area_ratio=HOLE_TO_CANOPY_AREA_RATIO, drag_coefficient=DRAG_COEFFICIENT)

radius = ParaCalc.calc_radius(circumference=circumference)

semi_ellipsoid_depth = ParaCalc.calc_semi_ellipsoid_depth(
    radius=radius, ratio_of_height_to_radius=RATIO_OF_HEIGHT_TO_RADIUS)

length_of_curve = ParaCalc.calc_length_of_curve(
    radius=radius, semi_ellipsoid_depth=semi_ellipsoid_depth, length_to_half_circumference_ratio=LENGTH_TO_HALF_CIRCUMFERENCE_RATIO)

length_of_curve_without_hole = ParaCalc.calc_length_of_curve_without_hole(
    length_of_curve=length_of_curve, hole_to_canopy_area_ratio=HOLE_TO_CANOPY_AREA_RATIO)

circular_area_of_selected = ParaCalc.calc_circular_area(
    area_cd=main_area_cd, drag_coefficient=DRAG_COEFFICIENT, hole_to_canopy_area_ratio=HOLE_TO_CANOPY_AREA_RATIO)

circular_diameter_of_selected = ParaCalc.calc_circular_diameter(
    circular_area_of_selected=circular_area_of_selected)

material_area_per_gore = ParaCalc.calc_material_area_per_gore(
    side_length=side_length, length_of_curve_without_hole=length_of_curve_without_hole)

material_area_per_chute = ParaCalc.calc_material_area_per_chute(
    number_of_gores_panels=NUMBER_OF_GORES_PANELS, material_area_per_gore=material_area_per_gore)

material_area_for_net = ParaCalc.calc_material_area_for_net(
    number_of_gores_panels=NUMBER_OF_GORES_PANELS, length_of_curve_without_hole=length_of_curve_without_hole, side_length=side_length)

mass_per_chute = ParaCalc.calc_mass_per_chute(
    material_area_per_chute=material_area_per_chute, mass_per_unit_area=mass_per_unit_area_SI)

cost_per_chute = ParaCalc.calc_cost_per_chute(
    material_area_for_net=material_area_for_net, cost_per_unit_area=cost_per_unit_area)


print(side_length)
print(circumference)
print(radius)
print(semi_ellipsoid_depth)
print(length_of_curve)
print(length_of_curve_without_hole)

print(circular_area_of_selected)
print(circular_diameter_of_selected)
print(material_area_per_gore)
print(material_area_per_chute)
print(material_area_for_net)
print(mass_per_chute)
print(cost_per_chute)

[0.50086167 0.50086167 0.50086167 0.50086167 0.50086167 0.16695389
 0.16695389 0.16695389 0.16695389 0.16695389]
2.0034466805560704
0.31885844243154804
[0.31885844 0.2869726  0.25508675 0.22320091 0.19131507 0.31885844
 0.2869726  0.25508675 0.22320091 0.19131507]
[0.50086167 0.47614816 0.45216786 0.42905342 0.40697476 0.50086167
 0.47614816 0.45216786 0.42905342 0.40697476]
[0.4808272  0.45710223 0.43408115 0.41189129 0.39069577 0.4808272
 0.45710223 0.43408115 0.41189129 0.39069577]
13.798423182026097
4.19150307769731
[0.15331581 0.14575091 0.13841044 0.13133501 0.12457664 0.05110527
 0.04858364 0.04613681 0.04377834 0.04152555]
[0.61326325 0.58300362 0.55364175 0.52534005 0.49830657 0.61326325
 0.58300362 0.55364175 0.52534005 0.49830657]
[0.96331166 0.91577995 0.86965843 0.82520223 0.78273814 0.96331166
 0.91577995 0.86965843 0.82520223 0.78273814]
[0.06439264 0.06121538 0.05813238 0.05516071 0.05232219 0.06439264
 0.06121538 0.05813238 0.05516071 0.05232219]
[5.77986999 5.49467969

In [20]:
round_parameters_dict = {
    'number_of_gores_panels': NUMBER_OF_GORES_PANELS,
    'ratio_of_height_to_radius': RATIO_OF_HEIGHT_TO_RADIUS,
    'drag_coefficient': DRAG_COEFFICIENT,
    'side_length': side_length,
    'circumference': circumference,
    'radius': radius,
    'length_of_curve': length_of_curve,
    'semi_ellipsoid_depth': semi_ellipsoid_depth,
    'length_of_curve_without_hole': length_of_curve_without_hole,
    'material_area_per_gore': material_area_per_gore,
    'material_area_per_chute': material_area_per_chute,
    'material_area_for_net': material_area_for_net,
    'mass_per_chute': mass_per_chute,
    'cost_per_chute': cost_per_chute
}

# Convert the dictionary to a DataFrame
# Convert the dictionary to a DataFrame
round_parameters_drogue_df = pd.DataFrame(round_parameters_dict)
display(round_parameters_drogue_df)

,number_of_gores_panels,ratio_of_height_to_radius,drag_coefficient,side_length,circumference,radius,length_of_curve,semi_ellipsoid_depth,length_of_curve_without_hole,material_area_per_gore,material_area_per_chute,material_area_for_net,mass_per_chute,cost_per_chute
0,4,1.0,1.5,0.500862,2.003447,0.318858,0.500862,0.318858,0.480827,0.153316,0.613263,0.963312,0.064393,5.779870
1,4,0.9,1.5,0.500862,2.003447,0.318858,0.476148,0.286973,0.457102,0.145751,0.583004,0.915780,0.061215,5.494680
2,4,0.8,1.5,0.500862,2.003447,0.318858,0.452168,0.255087,0.434081,0.138410,0.553642,0.869658,0.058132,5.217951
3,4,0.7,1.5,0.500862,2.003447,0.318858,0.429053,0.223201,0.411891,0.131335,0.525340,0.825202,0.055161,4.951213
4,4,0.6,1.5,0.500862,2.003447,0.318858,0.406975,0.191315,0.390696,0.124577,0.498307,0.782738,0.052322,4.696429
5,12,1.0,1.5,0.166954,2.003447,0.318858,0.500862,0.318858,0.480827,0.051105,0.613263,0.963312,0.064393,5.779870
6,12,0.9,1.5,0.166954,2.003447,0.318858,0.476148,0.286973,0.457102,0.048584,0.583004,0.915780,0.061215,5.494680
7,12,0.8,1.5,0.166954,2.003447,0.318858,0.452168,0.255087,0.434081,0.046137,0.553642,0.869658,0.058132,5.217951
8,12,0.7,1.5,0.166954,2.003447,0.318858,0.429053,0.223201,0.411891,0.043778,0.525340,0.825202,0.055161,4.951213
9,12,0.6,1.5,0.166954,2.003447,0.318858,0.406975,0.191315,0.390696,0.041526,0.498307,0.782738,0.052322,4.696429
